In [3]:
import pandas as pd
df = pd.read_csv('..\data.csv')
df.head()


,fpath,subject,payload,contenttype,isSpam
0,C:\Users\research_laptop\Desktop\projects\scho...,Re: New Sequences Window,"Date: Wed, 21 Aug 2002 10:54:46 -05...",text/plain,0.0
1,C:\Users\research_laptop\Desktop\projects\scho...,[zzzzteana] RE: Alexander,"Martin A posted:\nTassos Papadopoulos, the Gre...",text/plain,0.0
2,C:\Users\research_laptop\Desktop\projects\scho...,[zzzzteana] Moscow bomber,Man Threatens Explosion In Moscow \n\nThursday...,text/plain,0.0
3,C:\Users\research_laptop\Desktop\projects\scho...,[IRR] Klez: The Virus That Won't Die,Klez: The Virus That Won't Die\n \nAlready the...,text/plain,0.0
4,C:\Users\research_laptop\Desktop\projects\scho...,Re: [zzzzteana] Nothing like mama used to make,"> in adding cream to spaghetti carbonara, whi...",text/plain,0.0


In [14]:
df.dtypes


fpath           object
subject         object
payload         object
contenttype     object
isSpam         float64
dtype: object

In [36]:
remove = ['\n', '\t', '\r','\\n', '\\t', '\\r', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '-', '_', '+', '+', '{', '}', '[', ']', '\\', '|', ':', ';', '"', "'", ',', '.', '/', '?', '<', '>', '`', '~']
tmp = df['payload']
for r in remove:
    tmp = tmp.str.replace(r, '')
df['payload'] = tmp

C:\Users\research_laptop\AppData\Local\Temp\ipykernel_35356\3978132860.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  tmp = tmp.str.replace(r, '')
C:\Users\research_laptop\AppData\Local\Temp\ipykernel_35356\3978132860.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tmp = tmp.str.replace(r, '')


In [38]:
df.head()

,fpath,subject,payload,contenttype,isSpam
0,C:\Users\research_laptop\Desktop\projects\scho...,Re: New Sequences Window,Date Wed 21 Aug 2002 105446 0500 ...,text/plain,0.0
1,C:\Users\research_laptop\Desktop\projects\scho...,[zzzzteana] RE: Alexander,Martin A postedTassos Papadopoulos the Greek s...,text/plain,0.0
2,C:\Users\research_laptop\Desktop\projects\scho...,[zzzzteana] Moscow bomber,Man Threatens Explosion In Moscow Thursday Aug...,text/plain,0.0
3,C:\Users\research_laptop\Desktop\projects\scho...,[IRR] Klez: The Virus That Won't Die,Klez The Virus That Wont Die Already the most ...,text/plain,0.0
4,C:\Users\research_laptop\Desktop\projects\scho...,Re: [zzzzteana] Nothing like mama used to make,in adding cream to spaghetti carbonara which...,text/plain,0.0


In [40]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(lowercase=True, stop_words='english')
cv_data = vect.fit_transform(df['payload'])
cv_data

<9353x189101 sparse matrix of type '<class 'numpy.int64'>'
	with 1235764 stored elements in Compressed Sparse Row format>

In [53]:
count_vect_df = pd.DataFrame(cv_data.todense(), columns=vect.get_feature_names_out())
combined_df = pd.concat([df, count_vect_df], axis=1)

In [54]:
combined_df.drop(['fpath', 'subject'], axis=1, inplace=True)
combined_df.head()

,payload,contenttype,isSpam,00,000,0000,00000,000000,000000000000000000000000000032526amp3b223bbon203d2027273bwith2,000000000000000000000365000000000000000000000000000000000000000003640000,...,þàïáö¼ò,þë,þîñ,þõµ,þö,þööööà,þüg,ÿÿ,ÿÿã,ÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿò
0,Date Wed 21 Aug 2002 105446 0500 ...,text/plain,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Martin A postedTassos Papadopoulos the Greek s...,text/plain,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Man Threatens Explosion In Moscow Thursday Aug...,text/plain,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Klez The Virus That Wont Die Already the most ...,text/plain,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,in adding cream to spaghetti carbonara which...,text/plain,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [58]:
y = combined_df['isSpam']
x = combined_df.drop('isSpam', axis=1)


In [64]:
error_rate = []
knn = NearestNeighbors(n_neighbors=5)

#knn.fit(X_train, y_train)
#pred = knn.predict(X_test)
#error_rate.append(np.mean(pred != y_test))

In [78]:
tmp = combined_df.sample(frac=0.8)
y_train = tmp['isSpam']
x_train = tmp.drop(['isSpam', 'payload', 'contenttype'], axis=1)


In [80]:
tmp2 = combined_df[~combined_df.index.isin(tmp)]
y_test = tmp2['isSpam']
x_test = tmp2.drop(['isSpam', 'payload', 'contenttype'], axis=1)

In [81]:
knn.fit(x_train, y_train)
knn.
error_rate.append(np.mean(pred != y_test))

AttributeError: 'NearestNeighbors' object has no attribute 'predict'

In [86]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
sil = []
kmax = 3
for k in range(2, kmax):
    kmeans = KMeans(n_clusters = k).fit(x_train)
    labels = kmeans.labels_
    sil.append(silhouette_score(x_train, labels, metric = 'euclidean'))

In [89]:
sil = []
tmp = combined_df.drop(['isSpam', 'payload', 'contenttype'], axis=1)
kmeans = KMeans(n_clusters = 5).fit(tmp)
labels = kmeans.labels_
sil.append(silhouette_score(tmp, labels, metric = 'euclidean'))

In [97]:
sil = []
kmax = 50
for k in range(2, kmax):
    kmeans = KMeans(n_clusters=k).fit(tmp)
    labels = kmeans.labels_
    sil.append(silhouette_score(tmp,labels, metric='euclidean'))


In [88]:
sil.index(max(sil))
sil

[0.8813995044756892]

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(range(1,50),error_rate, marker='o', markersize=9)